## Neural Network completely from scratch
Implementation of a simple 4 layer neural network, and trained it on the MNIST digit recogniszer dataset.
I am going to use only numpy for calculations and the OneHotEncoder from sklearn just to One Hot Encode the labels.

In [2]:
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder

Load the dataset and split it into 90 and 10 percent.
90% for training and 10% for test

In [3]:
data = np.array(pd.read_csv("Datasets/digit-recognizer/train.csv"))
x_data, y_data = data[:, 1:data.shape[0]], data[:, 0:1]
indices = np.arange(len(x_data))
np.random.shuffle(indices)
x_data, y_data = x_data[indices], y_data[indices]

split = 0.9
idx = int(x_data.shape[0] * split)
trainx, trainy = x_data[:idx]/255., y_data[:idx]
x_validate, y_validate = x_data[idx:]/255., y_data[idx:]

## Step 1: Neural Network Architecture
The architecture will be: <br>
    <pre>Input layer: 784 neurons 
    Hidden layer 1: 16 neurons 
    Hidden layer 2: 16 neurons 
    Output layer: 10 neurons </pre>
So we will have 3 pairs of parameters i.e. <br>
Since we have 4 layers including input and output layer, we will have 3 pairs of parameters (weights and biases):
- **w1** *(16, 784)* and **b1** *(16, 1)* representing the weights and biases of the connections between the input layer *(784 neuron)* and the first hidden layer *(16 neuron).*
- **w2** *(16, 16)* and **b1** *(16, 1)* representing the weights and biases of the connections between the hidden layer 1 *(16 neuron)* and the hidden layer 2 *(16 neuron).*
- **w3** *(10, 16)* and **b1** *(10, 1)* representing the weights and biases of the connections between the hidden layer 2 *(16 neuron)* and the output layer *(10 neuron).*

In [4]:
def init_params():
    w1 = np.random.rand(16, 784) - 0.5
    b1 = np.random.rand(16, 1) - 0.5
    
    w2 = np.random.rand(16, 16) - 0.5
    b2 = np.random.rand(16, 1) - 0.5
    
    w3 = np.random.rand(10, 16) - 0.5
    b3 = np.random.rand(10, 1) - 0.5
    
    return w1, b1, w2, b2, w3, b3

## Step 2: Forward Propagation

### Activation Function
> Lets first define the activation functions, the activation function will simply filter out and activate only those neurons which will make a significant difference in the network

In [5]:
def activation(func: str, z, backprop=False):
    if func == 'relu':
        if backprop:
            return z > 0
        return np.maximum(0, z)

    elif func == 'sigmoid':
        return 1 / (1 + np.exp(-z))

    elif func == 'softmax':
        exp_z = np.exp(z - np.max(z, axis=0, keepdims=True))
        return exp_z / np.sum(exp_z, axis=0, keepdims=True)

### The main feed forward part
In forward propagation we take all of the activations from the previous layer and and computer their weighted sum according to their weight and then move on to the next layer to perform the same.

>  - From the input layer to first hidden layer
    $$Z_{1} = W_{1} \cdot X + b_{1}$$
    $$A_{1} = g_{\text{ ReLU}}(Z_{1}))$$
>  - First hidden layer to second hidden layer
    $$Z_{2} = W_{2} \cdot A_{1} + b_{2}$$
    $$A_{2} = g_{\text{ ReLU}}(Z_{2})$$
>  - Second hidden layer to Output layer
    $$Z_{3} = W_{3} \cdot A_{2} + b_{3}$$
    $$A_{3} = g_{\text{ softmax}}(Z_{3})$$

The output layers final activation *A_{3}* are the predictions for the given input, these predictions are in the form of probability ranging from (0-1)

In [6]:
def forward_prop(param, x):
    w1, b1, w2, b2, w3, b3 = param
    # input layer to hidden layer 1
    z1 = w1.dot(x) + b1
    a1 = activation('relu', z1)
    # hidden layer 1 to hidden layer 2
    z2 = w2.dot(a1) + b2
    a2 = activation('relu', z2)
    # hidden layer 2 to output layer
    z3 = w3.dot(a2) + b3
    a3 = activation('softmax', z3)
    
    return z1, a1, z2, a2, z3, a3

## Step 3: Loss / Error Calculation
Now we have a method to get prediction based on given inputs, now we need to measure how **wrong** our predictions are, we use a loss function to measure that. Basically we measure how much the predictions are deviating from the original labels.

In [7]:
def loss(func: str, y_predicted, labels):
    if func == 'cross_entropy':
        _, n = labels.shape
        epsilon = 1e-15
        y_predicted = np.maximum(epsilon, y_predicted)
        loss = -1/n * np.sum(labels * np.log(y_predicted))
        return loss

## Step 4: Learning Algorithm, Backpropagation
The final part of the neural network is to make it learn. For that we need to backpropagate the network and calculate the gradients, i.e. we calculate the gradients of output layer to the previous hidden layer, then of the hidden layer to the previous hidden layer ....
1. **Output layer (`W3`, `b3`) gradients:**
   $$dz3 = a3 - y$$
   $$dW3 = \frac{1}{m} \cdot dz3 \cdot a2^T$$
   $$db3 = \frac{1}{m} \cdot \text{sum}(dz3)$$
2. **Hidden layer 2 (`W2`, `b2`) gradients:**
   $$dz2 = W3^T \cdot dz3 \cdot \text{ReLU'}(z2)$$
   $$dW2 = \frac{1}{m} \cdot dz2 \cdot z1^T$$
   $$db2 = \frac{1}{m} \cdot \text{sum}(dz2)$$
3. **Hidden layer 1 (`W1`, `b1`) gradients:**
   $$dz1 = W2^T \cdot dz2 \cdot \text{ReLU'}(z1)$$
   $$dW1 = \frac{1}{m} \cdot dz1 \cdot x^T$$
   $$db1 = \frac{1}{m} \cdot \text{sum}(dz1)$$
**So finally we have the gradients `(dw1, db1), (dw2, db2), (dw3, db3)`**

In [8]:
def back_prop(zanda, params, x, y):
    m, _ = x.shape
    z1, a1, z2, a2, z3, a3 = zanda
    w1, b1, w2, b2, w3, b3 = params
    
    # Output layer gradients
    dz3 = a3 - y 
    dw3 = 1 / m * dz3.dot(a2.T)
    db3 = 1 / m * np.sum(dz3)
    # Hidden layer 2 gradient
    dz2 = w3.T.dot(dz3) * activation('relu', z2, backprop=True)
    dw2 = 1 / m * dz2.dot(z1.T)
    db2 = 1 / m * np.sum(dz2)
    # Hidden layer 1 gradient
    dz1 = w2.T.dot(dz2) * activation('relu', z1, backprop=True)
    dw1 = 1 / m * dz1.dot(x.T)
    db1 = 1 / m * np.sum(dz1)
    
    return dw1, db1, dw2, db2, dw3, db3

### Adjust the parameters to minimize the loss of the network
We have the gradients of each layer `(dw1, db1), (dw2, db2), (dw3, db3)`
Now we update the parameter by moving them closer to the global minima. We do it in small steps, that will be determined by some learning rate $\alpha$.
**Peform Gradient Descent**
1. For the output layer (W3, b3):
   $$W_3 -= \alpha \times dW_3$$
   $$b_3 -= \alpha \times db_3$$

2. For hidden layer 2 (W2, b2):
   $$W_2 -= \alpha \times dW_2$$
   $$b_2 -= \alpha \times db_2$$

3. For hidden layer 1 (W1, b1):
   $$W_1 -= \alpha \times dW_1$$
   $$b_1 -= \alpha \times db_1$$


In [9]:
def gradient_descent(param, gradient, learning_rate):
    w1, b1, w2, b2, w3, b3 = param
    dw1, db1, dw2, db2, dw3, db3 = gradient
    
    w1 -= learning_rate * dw1
    b1 -= learning_rate * db1
    w2 -= learning_rate * dw2
    b2 -= learning_rate * db2
    w3 -= learning_rate * dw3
    b3 -= learning_rate * db3
    
    return w1, b1, w2, b2, w3, b3

In [10]:
def accuracy(y_pred, y):
        predictions = np.argmax(y_pred, axis=0)
        acc = np.sum(predictions == y) / y.size
#         print(f"pp {predictions.shape}, {y.shape}, {y.size}")
        return acc

**We need to one hot encoded the lables for the loss calculation and gradients calculation, so we use the OneHotEncoder from Sklearn**

In [11]:
one_hot_y = OneHotEncoder().fit_transform(trainy).toarray()

##### **Now we use all of the methods to create the network and optimize the parameters, We train the network for 1000 epoch i.e. 1000 iteration on the given dataset.**
**And calculate and observe the loss and acuraccy for every 100 steps**

In [20]:
params = init_params()
epoch = 1000
learning_rate = 0.01
for i in range(epoch+1):
    zanda = forward_prop(params, trainx.T)
    gradients = back_prop(zanda, params, trainx.T, one_hot_y.T)
    params = gradient_descent(params, gradients, learning_rate)
    if i % 100 == 0:
        print(f"Epoch {i}/{epoch}")
        mloss = loss('cross_entropy', zanda[-1], one_hot_y.T)
        acc = accuracy(zanda[-1], trainy.T)
        print(f"loss: {mloss} - acc: {acc}")

Epoch 0/1000
loss: 3.67429459149603 - acc: 0.09526455026455026
Epoch 100/1000
loss: 0.5646364463315656 - acc: 0.8224867724867725
Epoch 200/1000
loss: 0.40460060787897173 - acc: 0.8705291005291005
Epoch 300/1000
loss: 0.34739063712420404 - acc: 0.8933333333333333
Epoch 400/1000
loss: 0.29898336558231814 - acc: 0.9082804232804232
Epoch 500/1000
loss: 0.2694288843430095 - acc: 0.9176455026455026
Epoch 600/1000
loss: 0.25413424900479853 - acc: 0.9219312169312169
Epoch 700/1000
loss: 0.24328867098892756 - acc: 0.9257671957671958
Epoch 800/1000
loss: 0.22327413517104425 - acc: 0.9316931216931217
Epoch 900/1000
loss: 0.2124299192578787 - acc: 0.9352380952380952
Epoch 1000/1000
loss: 0.22561748411273364 - acc: 0.9302910052910053


#### **Our neural network is giving 93% accuracy with low loss which is fair for something built from scratch now lets validate it by testing on some unseen data.**
**For testing we just need to do a forward pass through the network on the unseen data.**

In [21]:
def validate(x_val, y_val):
    enc_y = OneHotEncoder().fit_transform(y_val).toarray()
    _, _, _, _, _, a3 = forward_prop(params, x_val.T)
    vloss = loss('cross_entropy', a3, enc_y.T)
    acc = accuracy(a3, y_val.T)
    print(f"val_loss: {vloss} - val_acc: {acc}")

In [31]:
validate(x_validate, y_validate)

val_loss: 0.23088474886439597 - val_acc: 0.9369047619047619
